In [ ]:
import json
import re
from neo4j import GraphDatabase

# Credenciais de conexão com o Neo4j
url = "neo4j+s://XXXXXX.databases.neo4j.io"
usuario = "neo4j"
senha = "secret-key"

In [15]:
# Carregar os dados do arquivo JSON
with open('pokemon_data.json', 'r', encoding='utf-8') as file:
    pokemon_data = json.load(file)

In [ ]:
# Classe para criar o grafo dos Pokémons
class PokemonGraph:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def carrega_base(self, pokemon_data):
        with self.driver.session() as session:
            session.execute_write(self._zera_base)
            session.execute_write(self._create_graph, pokemon_data)

    @staticmethod
    def _zera_base(tx):
        tx.run("""
        MATCH (n)
        DETACH DELETE n
        """)

    @staticmethod
    def _create_graph(tx, pokemon_data):
        for pokemon in pokemon_data:
            # Corrigir o peso removendo caracteres especiais
            peso = float(re.sub(r'[^\d.]', '', pokemon['weight']))

            # Criar nós dos Pokémons
            tx.run("""
            MERGE (p:POKEMON {
                nome: $nome, 
                numero: $numero, 
                especie: $especie, 
                altura: $altura, 
                peso: $peso
            })
            """, 
            nome=pokemon['name'], 
            numero=pokemon['number'], 
            especie=pokemon['species'], 
            altura=pokemon['height'], 
            peso=peso)
            
            # Criar tipos de Pokémon
            for tipo in pokemon['types']:
                tx.run("""
                MERGE (t:TIPO {nome: $tipo})
                MERGE (p:POKEMON {nome: $nome})
                MERGE (p)-[:TEM_TIPO]->(t)
                """, 
                tipo=tipo, 
                nome=pokemon['name'])

            # Criar relações de evolução
            if 'evolution' in pokemon and isinstance(pokemon['evolution'], list):
                for evolucao in pokemon['evolution']:
                    tx.run("""
                    MERGE (p:POKEMON {nome: $nome})
                    MERGE (e:POKEMON {numero: $numero_evolucao, nome: $nome_evolucao})
                    MERGE (p)-[:EVOLUI_PARA]->(e)
                    """,
                    nome=pokemon['name'],
                    numero_evolucao=evolucao['number'],
                    nome_evolucao=evolucao['name'])

            # Criar habilidades
            if 'abilities' in pokemon and isinstance(pokemon['abilities'], list):
                for habilidade in pokemon['abilities']:
                    tx.run("""
                    MERGE (p:POKEMON {nome: $nome})
                    MERGE (h:HABILIDADE {nome: $nome_habilidade})
                    MERGE (p)-[:TEM_HABILIDADE]->(h)
                    """,
                    nome=pokemon['name'],
                    nome_habilidade=habilidade['name'])

# Executar o código para carregar o grafo
db = PokemonGraph(url, usuario, senha)
db.carrega_base(pokemon_data)
db.close()
print("Base de dados carregada")

Base de dados carregada


In [24]:
# Classe de conexão e consultas no Neo4j
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response


In [25]:
# Função de consulta
def consultar(query):
    conn = Neo4jConnection(uri=url, user=usuario, pwd=senha)
    resultado = conn.query(query)
    conn.close()
    return resultado

In [26]:
# Consultas
# Quais Pokémon podem atacar um Pikachu pelo sua fraqueza (tipo que lhe pode causar mais dano) cujo o peso é mais de 10kg?
print("Pokémons que podem atacar um Pikachu baseado na sua fraqueza e peso maior que 10kg:")
resultado1 = consultar("""
MATCH (p:POKEMON)-[:TEM_TIPO]->(t:TIPO)
WHERE t.nome IN ['Ground', 'Psychic'] AND p.peso > 10
RETURN p.nome AS Pokemon
""")
for registro in resultado1:
    print(registro['Pokemon'])

Pokémons que podem atacar um Pikachu baseado na sua fraqueza e peso maior que 10kg:
Nidoking
Iron Treads
Ting-Lu
Sandy Shocks
Great Tusk
Clodsire
Toedscruel
Ursaluna
Runerigus
Sandaconda
Palossand
Mudsdale
Mudbray
Diggersby
Landorus
Golurk
Golett
Stunfisk
Krookodile
Krokorok
Sandile
Seismitoad
Palpitoad
Excadrill
Mamoswine
Rhyperior
Gliscor
Hippowdon
Hippopotas
Garchomp
Gabite
Gible
Gastrodon
Torterra
Groudon
Claydol
Baltoy
Whiscash
Flygon
Vibrava
Numel
Trapinch
Camerupt
Swampert
Marshtomp
Pupitar
Larvitar
Donphan
Phanpy
Piloswine
Steelix
Gligar
Quagsire
Rhydon
Rhyhorn
Marowak
Onix
Golem
Nidoqueen
Sandslash
Sandshrew
Dugtrio
Iron Crown
Veluza
Farigiraf
Espathra
Wyrdeer
Mr. Rime
Orbeetle
Dottler
Cosmoem
Tapu Lele
Lunala
Solgaleo
Bruxish
Oranguru
Malamar
Delphox
Braviary
Beheeyem
Reuniclus
Sigilyph
Gothitelle
Swoobat
Musharna
Cresselia
Gallade
Mime Jr.
Bronzong
Bronzor
Deoxys
Latios
Latias
Metagross
Metang
Beldum
Wynaut
Claydol
Baltoy
Solrock
Lunatone
Grumpig
Spoink
Gardevoir
Lugia
Giraf

In [22]:
# Qual é o tipo mais comum de um Pokémon que é atacado pelo tipo gelo?
print("\nTipo mais comum de um Pokémon que é atacado pelo tipo Gelo:")
resultado2 = consultar("""
MATCH (p:POKEMON)-[:TEM_TIPO]->(t:TIPO)
WHERE t.nome = 'Ice'
RETURN t.nome AS Tipo, COUNT(*) AS Quantidade
ORDER BY Quantidade DESC
LIMIT 1
""")
for registro in resultado2:
    print(registro['Tipo'])


Tipo mais comum de um Pokémon que é atacado pelo tipo Gelo:
Ice


In [20]:
# Quantas segundas e terceiras evoluções que fazem um Pokémon no mínimo dobrar de peso?
print("\nQuantidade de segundas e terceiras evoluções que fazem um Pokémon dobrar de peso:")
resultado3 = consultar("""
MATCH (p1:POKEMON)-[:EVOLUI_PARA]->(p2:POKEMON)
WHERE p2.peso >= p1.peso * 2
RETURN COUNT(*) AS Total
""")
for registro in resultado3:
    print(registro['Total'])



Quantidade de segundas e terceiras evoluções que fazem um Pokémon dobrar de peso:
344
